In [1]:
import networkx as nx
from numpy.core.defchararray import title
import torch
import dgl
import matplotlib.pyplot as plt
coauthors, _ = dgl.load_graphs(f'../save2/coauthors_topic.graph')
i = 0

g = coauthors[i]
edge_topic = g.edata['topic']
idx = torch.where(edge_topic.sum(1)!=0)[0]
new_g = dgl.edge_subgraph(g, idx, preserve_nodes=False)
G = dgl.to_networkx(new_g, edge_attrs=['topic'])
G = nx.Graph(G)

Using backend: pytorch


In [4]:
import numpy as np
## 1. 基础拓扑特征
# a. |V|, |E|, degree：平均和最大
E = G.number_of_edges()
V = G.number_of_nodes()
degree = [d for _, d in G.degree()]
d_avg = np.mean(degree)
d_max = np.max(degree)
# b. 最短路径：分布、平均
shortest_path_len = dict(nx.all_pairs_shortest_path_length(G))
l_avg = np.mean([np.mean(list(spl.values())) for spl in shortest_path_len.values()])
# draw_spl(shortest_path_len, path=f'../jpg/spl_{i}.jpg')
# c. 密度
ρ = nx.density(G)
# d. 连通分量个数
n_cc = nx.number_connected_components(G)
# e. 社区个数：社区是一组节点，组内的节点连接的边比组之间的多得多。
communities = nx.community.label_propagation_communities(G)
n_c = len(list(communities))
print(V, E, d_max, d_avg, l_avg, ρ, n_cc, n_c)

8957 9628 17 2.149826950988054 4.562168123312704 0.00024004320578249822 2053 2738


In [6]:
## 3. Triangles
triangles = nx.triangles(G)
T_top8 = (sorted(triangles.items(), key=lambda item: item[1], reverse=True))[:8]
triangles_per_node = list(triangles.values())
T = sum(triangles_per_node) / 3
T_avg = np.mean(triangles_per_node)
T_max = np.max(triangles_per_node)

## 4. Coefficient系数
# a. Assort. Coeff. 分类系数描述了网络节点以某种方式附加到其他节点的偏好
r = nx.degree_assortativity_coefficient(G)
# b. 聚类系数：两个随机选择的朋友彼此成为朋友的概率，是三元闭包的标志
K = nx.clustering(G)
K_avg = nx.average_clustering(G)
K_top8 = (sorted(K.items(), key=lambda item: item[1], reverse=True))[:8]
# draw_hist(K, path=f'../jpg/K_{i}.jpg', title='Clustering Coefficient')

## 5. Maximum Cliques 最大团: 包含v的最大完全子图
max_cliq = list(nx.find_cliques_recursive(G))
w_num = len(max_cliq)
w_lb = min([len(nc) for nc in max_cliq])
w_hb = max([len(nc) for nc in max_cliq])

## 6. Bridges: 删除边会导致 A 和 B 位于不同的分量中
bridges = list(nx.bridges(G))
local_bridges = list(nx.local_bridges(G, with_span=False))
n_b, n_lb = len(bridges), len(local_bridges)

kcore = list(nx.k_core(G))
K_max = max(kcore)

print(V, E, d_max, d_avg, l_avg, ρ, n_cc, n_c, \
        # dc, bc, cc, ec, \
        T, T_avg, T_max,\
        r, K_avg, \
        w_num, w_lb, w_hb, \
        n_b, n_lb, K_max)

8957 9628 17 2.149826950988054 4.562168123312704 0.00024004320578249822 2053 2738 3685.0 1.23423021100815 37 0.3819855058064926 0.3306309940102779 5311 2 10 3622 3723 8025
